In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import urllib

server = 'auqui.database.windows.net'
port = '1433'
database = 'dbnormativo'
username = 'administrador'
password = 'Auqui$2020'
driver = '{ODBC Driver 17 for SQL Server}'                     

connection = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)
conn = f"""Driver={driver};Server=tcp:{server},1433;Database={database};
Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"""

params = urllib.parse.quote_plus(conn)
conn_str = 'mssql+pyodbc:///?autocommit=true&odbc_connect={}'.format(params)

In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import sqlalchemy
from sqlalchemy import create_engine

class PRC_RCD_REFINANCIAMIENTO():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d')
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def ProcesaClasificacionRefinanciado(self):
    
    fecha_proceso_anterior = self.fecha_proceso.replace(day=1) - timedelta(days=1)
    fecha_proceso_anterior_str = fecha_proceso_anterior.strftime('%Y-%m-%d')
    
    #actualizando la clasificacion del mes anterior
    UpdateRefDiasAnterior = """
    UPDATE RD
    SET RD.CLASIFICACION_DEUDOR = CASE WHEN CLA.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO IS NULL THEN B.CLASIFICACION_DEUDOR
                                  ELSE
                                      CASE
                                      WHEN CLA.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO = '0' THEN '1' 
                                      ELSE CLA.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO END
                                  END
    FROM CLIENTE CL
    JOIN CLIENTE_PRODUCTO CP
    ON CP.CLIENTE = CL.ID
    JOIN REFINANCIAMIENTO R
    ON R.CLIENTE_DEUDOR_PRODUCTO = CP.CLIENTE_DEUDOR_PRODUCTO
    JOIN REFINANCIAMIENTO_DIAS RD
    ON RD.REFINANCIAMIENTO = R.ID
    JOIN CLASIFICACION_DEUDOR_DIAS_ATRASO B
    ON CP.TIPO_CREDITO_SBS = B.TIPO_CREDITO_SBS
    AND R.DIAS_ATRASO_ANTES_REF BETWEEN B.DIAS_ATRASO_MINIMO AND ISNULL(B.DIAS_ATRASO_MAXIMO,R.DIAS_ATRASO_ANTES_REF+1)
    LEFT JOIN CLIENTE_PRODUCTO CPA
    ON CPA.CLIENTE_DEUDOR_PRODUCTO = R.CLIENTE_DEUDOR_PRODUCTO
    LEFT JOIN CLIENTE CLA
    ON CLA.ID = CPA.CLIENTE
    AND CLA.FECHA_PROCESO = """"'"+ fecha_proceso_anterior_str + """'
    WHERE R.FECHA_PROCESO = """"'"+ self.fecha_proceso_str + """'
    AND RD.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    AND CL.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    """
    self.cursor.execute(UpdateRefDiasAnterior)
    
    updateRefAnterior = """
    UPDATE R
    SET R.CLASIFICACION_DEUDOR = RD.CLASIFICACION_DEUDOR
    FROM REFINANCIAMIENTO R
    JOIN REFINANCIAMIENTO_DIAS RD
    ON RD.REFINANCIAMIENTO = R.ID
    WHERE R.FECHA_PROCESO = """"'"+ self.fecha_proceso_str + """'
    AND RD.FECHA_PROCESO = """"'"+ self.fecha_proceso_str + """'
    """
    self.cursor.execute(updateRefAnterior)
    self.connection.commit() 
    
    #EMPEORAMIENTO DE CLASIFICACION
    refDiasAnterior = ("SELECT RD.REFINANCIAMIENTO, RD.CLASIFICACION_DEUDOR FROM REFINANCIAMIENTO_DIAS RD WHERE RD.FECHA_PROCESO = '" + fecha_proceso_anterior_str + "'")
    sql_refDiasAnterior = pd.read_sql_query(refDiasAnterior, self.connection)
    df_refDiasAnterior = pd.DataFrame(sql_refDiasAnterior)
    
    diasAtraso = """SELECT RD.FECHA_PROCESO, RD.REFINANCIAMIENTO, CP.DIAS_ATRASO
    FROM CLIENTE CL
    JOIN CLIENTE_PRODUCTO CP
    ON CP.CLIENTE = CL.ID
    JOIN REFINANCIAMIENTO R
    ON R.CLIENTE_DEUDOR_PRODUCTO = CP.CLIENTE_DEUDOR_PRODUCTO
    JOIN REFINANCIAMIENTO_DIAS RD
    ON RD.REFINANCIAMIENTO = R.ID
    WHERE CL.FECHA_PROCESO = """"'"+ self.fecha_proceso_str + """'
    AND RD.FECHA_PROCESO = """"'"+ self.fecha_proceso_str + """'
    """
    sql_diasAtraso = pd.read_sql_query(diasAtraso, self.connection)
    df_diasAtraso = pd.DataFrame(sql_diasAtraso)
    
    join_diasAtraso_refDias = pd.merge(df_refDiasAnterior,df_diasAtraso, on = ['REFINANCIAMIENTO'], how='inner')
    
    join_diasAtraso_refDias.to_sql('TEMP_DIASATRASOREF', con=conn_str, if_exists='replace',index=False)
    
    updaterefDias = """
    UPDATE RD
    SET RD.CLASIFICACION_DEUDOR = CASE WHEN X.DIAS_ATRASO > 0 AND X.CLASIFICACION_DEUDOR != '4' 
                                       THEN CAST(X.CLASIFICACION_DEUDOR AS INT) + 1 
                                  ELSE X.CLASIFICACION_DEUDOR END
    FROM REFINANCIAMIENTO_DIAS RD
    JOIN TEMP_DIASATRASOREF X
    ON RD.FECHA_PROCESO = X.FECHA_PROCESO
    AND RD.REFINANCIAMIENTO = X.REFINANCIAMIENTO
    """
    self.cursor.execute(updaterefDias)
    self.connection.commit()
    
    #MEJORAMIENTO DE CLASIFICACION
    #reproceso
    validaSemestrenull = """UPDATE RD
    SET RD.FECHA_VAL_TRIMESTRE = NULL
    FROM REFINANCIAMIENTO_DIAS RD
    WHERE RD.FECHA_VAL_TRIMESTRE = """"'"+ self.fecha_proceso_str + """'
    """
    self.cursor.execute(validaSemestrenull)
    
    #marcamos los 2 trimestres
    aplicaSemestre= """;WITH X AS(
    SELECT REFINANCIAMIENTO, COUNT(FECHA_PROCESO) AS CONTADOR
    FROM REFINANCIAMIENTO_DIAS
    WHERE FECHA_VAL_TRIMESTRE IS NULL
    GROUP BY REFINANCIAMIENTO
    HAVING COUNT(FECHA_PROCESO) = 7
    )
    UPDATE RD
    SET RD.FECHA_VAL_TRIMESTRE = """"'"+ self.fecha_proceso_str + """'
    FROM X
    JOIN REFINANCIAMIENTO_DIAS RD
    ON RD.REFINANCIAMIENTO = X.REFINANCIAMIENTO
    WHERE RD.FECHA_VAL_TRIMESTRE IS NULL
    AND RD.FECHA_PROCESO != """"'"+ self.fecha_proceso_str + """'
    """
    self.cursor.execute(aplicaSemestre)
    
    #actualizamos clasificacion a los que mejoran
    mejora = """;WITH X AS(
    SELECT REFINANCIAMIENTO
          ,SUM(DIAS_ATRASO) AS DIAS_ATRASO
    FROM REFINANCIAMIENTO_DIAS
    WHERE FECHA_VAL_TRIMESTRE = """"'"+ self.fecha_proceso_str + """'
    GROUP BY REFINANCIAMIENTO
    HAVING SUM(DIAS_ATRASO) = 0
    ), Y AS(
    SELECT RD.REFINANCIAMIENTO
          ,RD.CLASIFICACION_DEUDOR
    FROM REFINANCIAMIENTO_DIAS RD
    JOIN X
    ON X.REFINANCIAMIENTO = RD.REFINANCIAMIENTO
    WHERE RD.FECHA_PROCESO = """"'"+ fecha_proceso_anterior_str + """'
    )
    UPDATE RD
    SET RD.CLASIFICACION_DEUDOR = CASE WHEN CAST(Y.CLASIFICACION_DEUDOR AS INT) = 0 THEN CAST(Y.CLASIFICACION_DEUDOR AS INT) ELSE CAST(Y.CLASIFICACION_DEUDOR AS INT) - 1 END
    FROM Y
    JOIN REFINANCIAMIENTO_DIAS RD
    ON RD.REFINANCIAMIENTO = Y.REFINANCIAMIENTO
    WHERE RD.FECHA_PROCESO = """"'"+ self.fecha_proceso_str + """'
    """   
    self.cursor.execute(mejora)
    self.connection.commit()
    
    #actualiza clasificacion rcd
    updateClasif_cliProd = """UPDATE CP
    SET CP.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO = RD.CLASIFICACION_DEUDOR
    FROM CLIENTE CL
    JOIN CLIENTE_PRODUCTO CP
    ON CP.CLIENTE = CL.ID
    JOIN REFINANCIAMIENTO R
    ON R.CLIENTE_DEUDOR_PRODUCTO = CP.CLIENTE_DEUDOR_PRODUCTO
    JOIN REFINANCIAMIENTO_DIAS RD
    ON RD.REFINANCIAMIENTO = R.ID
    WHERE RD.FECHA_PROCESO = """"'"+ self.fecha_proceso_str + """'
    AND CL.FECHA_PROCESO = """"'"+ self.fecha_proceso_str + """'
    """
    self.cursor.execute(updateClasif_cliProd)
    
    updateClasif_cli = """UPDATE A SET A.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO = B.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO, 
    A.CLASIFICACION_DEUDOR = B.CLASIFICACION_DEUDOR_SIN_ALINEAMIENTO
    FROM CLIENTE A 
    JOIN CLIENTE_PRODUCTO B 
    ON A.ID = B.CLIENTE
    WHERE A.FECHA_PROCESO = """"'"+ self.fecha_proceso_str + """'
    """
    self.cursor.execute(updateClasif_cli)
    self.connection.commit()
      
#Ejecucion prueba
#PRC_RCD_REFINANCIAMIENTO = PRC_RCD_REFINANCIAMIENTO(1)
#PRC_RCD_REFINANCIAMIENTO.ProcesaClasificacionRefinanciado()